# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# Implement 
print("Total number of twits:", len(twits['data']))


Total number of twits: 1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http\S+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'\$\w*', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@\w*', ' ', text)

    # Replace everything not a letter with a space
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(word) for word in tokens if len(word) > 1]

    
    assert type(tokens) == list, 'Tokens should be list'
    return tokens

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rhelt\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


>Note: You must ensure that after preprocessing the text should NOT include:
- Numbers
- URLs
- Single character tokens
- Ticker symbols (these should be removed even if they don't appear at the beginning)

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement

tokenized = [preprocess(message) for message in messages]

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 

bow = Counter([word for tokens in tokenized for word in tokens])

bow

Counter({'the': 398809,
         'to': 379508,
         'is': 284887,
         'for': 273544,
         'on': 241666,
         'of': 211336,
         'and': 208508,
         'in': 205323,
         'this': 203572,
         'it': 193535,
         'at': 138459,
         'will': 128206,
         'up': 121577,
         'are': 101427,
         'you': 94325,
         'that': 89677,
         'be': 89281,
         'short': 86671,
         'what': 79141,
         'buy': 76326,
         'today': 76248,
         'stock': 75432,
         'here': 74957,
         'just': 73196,
         'down': 69368,
         'with': 68861,
         'not': 68308,
         'call': 67762,
         'day': 67487,
         'we': 64757,
         'from': 62387,
         'all': 59895,
         'if': 59001,
         'now': 57534,
         'out': 56703,
         'they': 56357,
         'more': 55368,
         'report': 55267,
         'wa': 55257,
         'by': 53983,
         'market': 53671,
         'but': 53512,
         

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the **number of messages** calculated in the code block above (i.e. `len(messages))`. Then remove words that only appear in some small fraction of the messages.

>Note: There is not an exact number for low and high-frequency cut-offs, however there is a correct optimal range.
You should ideally set up low-frequency cut-off from 0.0000002 to 0.000007 (inclusive) and high-frequency from 5 to 20 (inclusive). If the number is too big, we lose lots of important words that we can use in our data.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word: count/len(tokenized) for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.000001 

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 15

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [word for word, count in bow.most_common(high_cutoff)]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['the', 'to', 'is', 'for', 'on', 'of', 'and', 'in', 'this', 'it', 'at', 'will', 'up', 'are', 'you']


48002

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: ii for ii, word in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: word for word, ii in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in message if word in vocab] for message in tokenized]

assert set(vocab.keys()) == set(id2vocab.values()), 'Check vocab and id2vocab dictionaries'

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [10]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [11]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1948055349680063

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [13]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        
        # Setup additional layers
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size, output_size)
        self.log_softmax = nn.LogSoftmax(dim=1)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        # Debugging statement
        # print(f"Input size: {nn_input.size()}")
        embeds = self.embedding(nn_input)
        lstm_out, hidden_state = self.lstm(embeds, hidden_state)
        out = self.dropout(lstm_out)
        out = self.fc(out[-1])
        return self.log_softmax(out), hidden_state


### View Model

In [14]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.7345, -1.6314, -1.7305, -1.5352, -1.4471],
        [-1.7464, -1.6172, -1.7293, -1.5411, -1.4456],
        [-1.7183, -1.6660, -1.7257, -1.5124, -1.4558],
        [-1.7309, -1.6826, -1.7264, -1.5270, -1.4194]],
       grad_fn=<LogSoftmaxBackward0>)


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [15]:
import torch
import random

def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_end = min(ii + batch_size, total_sequences)
        batch_messages = messages[ii:batch_end]
        batch_labels = labels[ii:batch_end]

        # Ensure each message has a corresponding label
        if len(batch_messages) != len(batch_labels):
            continue  # Skip this batch if sizes don't match

        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]

        label_tensor = torch.tensor(batch_labels, dtype=torch.int64)

        # Debugging statements
        # print(f"Input size: {batch.size()}")
        # print(f"Batch size (messages): {len(batch_messages)}")
        # print(f"Batch size (labels): {len(label_tensor)}")

        yield batch, label_tensor


### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [16]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

split_idx = int(len(token_ids) * 0.8)  # 80% of data for training
train_features = token_ids[:split_idx]
valid_features = token_ids[split_idx:]

train_labels = sentiments[:split_idx]
valid_labels = sentiments[split_idx:]

In [17]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(48003, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (log_softmax): LogSoftmax(dim=1)
)

In [19]:
import torch
from torch import nn, optim

def validate_model(model, criterion, dataloader, valid_features, valid_labels, batch_size):
    """
    Validate the model on validation dataset.
    """
    val_losses = []
    val_accuracies = []
    model.eval()
    
    with torch.no_grad():
        for val_text_batch, val_labels in dataloader(valid_features, valid_labels, batch_size=batch_size, sequence_length=20):
            val_text_batch, val_labels = val_text_batch.to(device), val_labels.to(device)
            val_hidden = model.init_hidden(val_text_batch.size(1))
            val_hidden = tuple(each.data for each in val_hidden)
            
            val_logps, _ = model(val_text_batch, val_hidden)
            val_loss = criterion(val_logps.squeeze(), val_labels)
            val_losses.append(val_loss.item())
            
            ps = torch.exp(val_logps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == val_labels.view(*top_class.shape)
            val_accuracies.append(torch.mean(equals.type(torch.FloatTensor)).item())
    
    avg_val_loss = sum(val_losses) / len(val_losses)
    avg_val_accuracy = sum(val_accuracies) / len(val_accuracies)
    return avg_val_loss, avg_val_accuracy

# Initialize training parameters
epochs = 3
batch_size = 64
learning_rate = 0.001
print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print("Starting Training...")

# Training loop
for epoch in range(epochs):
    print(f'Starting epoch {epoch + 1}/{epochs}')
    steps = 0
    running_loss = 0
    model.train()
    
    # Training each batch
    for text_batch, labels in dataloader(train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(text_batch.size(1))
        hidden = tuple(each.data for each in hidden)  # Detach hidden state
        
        text_batch, labels = text_batch.to(device), labels.to(device)
        optimizer.zero_grad()
        logps, hidden = model(text_batch, hidden)
        loss = criterion(logps.squeeze(), labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        # Validation step
        if steps % print_every == 0:
            val_loss, val_accuracy = validate_model(model, criterion, dataloader, valid_features, valid_labels, batch_size)
            model.train()
            print(f"Epoch: {epoch + 1}/{epochs}.. Step: {steps}.. Loss: {running_loss/print_every:.3f}.. Val Loss: {val_loss:.3f}.. Val Accuracy: {val_accuracy:.3f}")
            running_loss = 0

    print(f'Epoch {epoch + 1}/{epochs} completed.')

print("Training Complete")


Starting Training...
Starting epoch 1/3
Epoch: 1/3.. Step: 100.. Loss: 1.330.. Val Loss: 1.234.. Val Accuracy: 0.501
Epoch: 1/3.. Step: 200.. Loss: 1.140.. Val Loss: 1.128.. Val Accuracy: 0.543
Epoch: 1/3.. Step: 300.. Loss: 1.052.. Val Loss: 1.074.. Val Accuracy: 0.567
Epoch: 1/3.. Step: 400.. Loss: 1.037.. Val Loss: 1.014.. Val Accuracy: 0.600
Epoch: 1/3.. Step: 500.. Loss: 0.946.. Val Loss: 0.983.. Val Accuracy: 0.610
Epoch: 1/3.. Step: 600.. Loss: 0.931.. Val Loss: 0.954.. Val Accuracy: 0.625
Epoch: 1/3.. Step: 700.. Loss: 0.923.. Val Loss: 0.937.. Val Accuracy: 0.637
Epoch: 1/3.. Step: 800.. Loss: 0.898.. Val Loss: 0.936.. Val Accuracy: 0.632
Epoch: 1/3.. Step: 900.. Loss: 0.881.. Val Loss: 0.913.. Val Accuracy: 0.641
Epoch: 1/3.. Step: 1000.. Loss: 0.883.. Val Loss: 0.895.. Val Accuracy: 0.654
Epoch: 1/3.. Step: 1100.. Loss: 0.864.. Val Loss: 0.895.. Val Accuracy: 0.651
Epoch: 1/3.. Step: 1200.. Loss: 0.868.. Val Loss: 0.871.. Val Accuracy: 0.663
Epoch: 1/3.. Step: 1300.. Loss: 0

Epoch: 1/3.. Step: 10600.. Loss: 0.704.. Val Loss: 0.729.. Val Accuracy: 0.720
Epoch: 1/3.. Step: 10700.. Loss: 0.717.. Val Loss: 0.733.. Val Accuracy: 0.720
Epoch: 1/3.. Step: 10800.. Loss: 0.727.. Val Loss: 0.731.. Val Accuracy: 0.720
Epoch: 1/3.. Step: 10900.. Loss: 0.749.. Val Loss: 0.731.. Val Accuracy: 0.721
Epoch: 1/3.. Step: 11000.. Loss: 0.706.. Val Loss: 0.729.. Val Accuracy: 0.722
Epoch: 1/3.. Step: 11100.. Loss: 0.720.. Val Loss: 0.727.. Val Accuracy: 0.722
Epoch: 1/3.. Step: 11200.. Loss: 0.700.. Val Loss: 0.731.. Val Accuracy: 0.721
Epoch: 1/3.. Step: 11300.. Loss: 0.722.. Val Loss: 0.725.. Val Accuracy: 0.722
Epoch: 1/3.. Step: 11400.. Loss: 0.718.. Val Loss: 0.727.. Val Accuracy: 0.722
Epoch: 1/3.. Step: 11500.. Loss: 0.724.. Val Loss: 0.730.. Val Accuracy: 0.718
Epoch: 1/3.. Step: 11600.. Loss: 0.720.. Val Loss: 0.731.. Val Accuracy: 0.719
Epoch: 1/3.. Step: 11700.. Loss: 0.730.. Val Loss: 0.722.. Val Accuracy: 0.723
Epoch: 1/3.. Step: 11800.. Loss: 0.737.. Val Loss: 0

Epoch: 2/3.. Step: 8300.. Loss: 0.686.. Val Loss: 0.723.. Val Accuracy: 0.723
Epoch: 2/3.. Step: 8400.. Loss: 0.656.. Val Loss: 0.723.. Val Accuracy: 0.725
Epoch: 2/3.. Step: 8500.. Loss: 0.682.. Val Loss: 0.721.. Val Accuracy: 0.723
Epoch: 2/3.. Step: 8600.. Loss: 0.671.. Val Loss: 0.725.. Val Accuracy: 0.721
Epoch: 2/3.. Step: 8700.. Loss: 0.674.. Val Loss: 0.718.. Val Accuracy: 0.724
Epoch: 2/3.. Step: 8800.. Loss: 0.667.. Val Loss: 0.717.. Val Accuracy: 0.726
Epoch: 2/3.. Step: 8900.. Loss: 0.688.. Val Loss: 0.720.. Val Accuracy: 0.722
Epoch: 2/3.. Step: 9000.. Loss: 0.672.. Val Loss: 0.717.. Val Accuracy: 0.724
Epoch: 2/3.. Step: 9100.. Loss: 0.692.. Val Loss: 0.718.. Val Accuracy: 0.725
Epoch: 2/3.. Step: 9200.. Loss: 0.684.. Val Loss: 0.719.. Val Accuracy: 0.721
Epoch: 2/3.. Step: 9300.. Loss: 0.690.. Val Loss: 0.716.. Val Accuracy: 0.726
Epoch: 2/3.. Step: 9400.. Loss: 0.689.. Val Loss: 0.716.. Val Accuracy: 0.726
Epoch: 2/3.. Step: 9500.. Loss: 0.663.. Val Loss: 0.721.. Val Ac

Epoch: 3/3.. Step: 6000.. Loss: 0.633.. Val Loss: 0.728.. Val Accuracy: 0.724
Epoch: 3/3.. Step: 6100.. Loss: 0.621.. Val Loss: 0.733.. Val Accuracy: 0.724
Epoch: 3/3.. Step: 6200.. Loss: 0.624.. Val Loss: 0.734.. Val Accuracy: 0.724
Epoch: 3/3.. Step: 6300.. Loss: 0.640.. Val Loss: 0.731.. Val Accuracy: 0.725
Epoch: 3/3.. Step: 6400.. Loss: 0.645.. Val Loss: 0.733.. Val Accuracy: 0.721
Epoch: 3/3.. Step: 6500.. Loss: 0.616.. Val Loss: 0.733.. Val Accuracy: 0.722
Epoch: 3/3.. Step: 6600.. Loss: 0.611.. Val Loss: 0.739.. Val Accuracy: 0.722
Epoch: 3/3.. Step: 6700.. Loss: 0.627.. Val Loss: 0.737.. Val Accuracy: 0.721
Epoch: 3/3.. Step: 6800.. Loss: 0.632.. Val Loss: 0.739.. Val Accuracy: 0.722
Epoch: 3/3.. Step: 6900.. Loss: 0.649.. Val Loss: 0.732.. Val Accuracy: 0.723
Epoch: 3/3.. Step: 7000.. Loss: 0.648.. Val Loss: 0.732.. Val Accuracy: 0.725
Epoch: 3/3.. Step: 7100.. Loss: 0.637.. Val Loss: 0.730.. Val Accuracy: 0.724
Epoch: 3/3.. Step: 7200.. Loss: 0.632.. Val Loss: 0.734.. Val Ac

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [20]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    # Convert words to ids
    tokens = [vocab[word] for word in tokens if word in vocab]

        
    # Adding a batch dimension
    text_input = torch.tensor(tokens).unsqueeze(1)
    # Get the NN output
    hidden = model.init_hidden(1)
    with torch.no_grad():
        logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.
    pred = torch.exp(logps)
    
    return pred

In [21]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

tensor([[1.8075e-04, 3.9881e-02, 1.2721e-02, 8.5338e-01, 9.3841e-02]])

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Given the output tensor `[[1.8075e-04, 3.9881e-02, 1.2721e-02, 8.5338e-01, 9.3841e-02]]`, we can interpret the model's prediction and its uncertainty as follows:

**Prediction of the Model:**
The prediction is given by the class with the highest probability. In this case, the largest value in the tensor is `8.5338e-01` (which is approximately 85.34%). This suggests that the model predicts the input text to be most strongly associated with the fourth class (index 3 if zero-indexed).

**Uncertainty of the Prediction:**
The uncertainty can be assessed by examining the spread of the probabilities. If one class has a significantly higher probability than the others, as in this case, the model is more confident in its prediction. Here, the fourth class has a much higher probability than the other classes, indicating a lower level of uncertainty.

To quantify the uncertainty, you could look at metrics like entropy or variance of the prediction probabilities. In this case, the entropy would be low because the probabilities are not spread evenly across the classes, and the variance would be high, showing that one class's probability is much higher than the rest, thus indicating a confident prediction.

To summarize, the model is confidently predicting the fourth class for the input text, with low uncertainty due to the high probability assigned to this class compared to the others.

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [22]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [23]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [24]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [25]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': tensor([[0.0479, 0.0676, 0.3042, 0.2975, 0.2829]]),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.